# Data management

## Case study: Management quality and firm size: describing patterns of association

## [Michel Coppée](https://www.uliege.be/cms/c_9054334/fr/repertoire?uid=u224042) & [Malka Guillot](https://malkaguillot.github.io/)

## HEC Liège | [ECON2306]()

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/malkaguillot/ECON2206-Data-Management-2023/HEAD?labpath=%2Fpractice%2F3.1-correlation.ipynb)

This case study aims to use data to study the relationship between firm management and firm size. 

Data: `wms-management-survey` dataset

Reference: *Data Analysis for Business, Economics, and Policy* by Gabor Bekes and  Gabor Kezdi, chapter 4 ( [website](https://gabors-data-analysis.com/) )

In [1]:
import os
import sys

import numpy as np
import pandas as pd
from mizani.formatters import percent_format
from plotnine import *

# Current script folder
current_path = os.getcwd()

dirname = current_path.split("practice")[0]
#print(dirname)

# location folders
data_in = dirname + "data/"
data_out = dirname + "practice/"
output = dirname + "practice/output/"

func = dirname + "practice/utils/"
sys.path.append(func)
from py_helper_functions_short import *

In [2]:
import warnings
warnings.filterwarnings("ignore")

### 1. Import the `wms_da_textbook.csv` data

In [ ]:
# Import data
df = pd.read_csv(data_in + "wms_da_textbook.csv")

<div class="alert alert-info">
<h4> Your turn</h4>
    Select the following observation for `df` (and replace `df` with the corresponding selection): 
    
    - Country = Mexico,
    - year = 2013
    - firms with a number of employees between 100 & 5000 (column name = `emp_firm`)
    
</div>


Summary of our 2 main variables of interest:

In [ ]:
df.filter(["management", "emp_firm"]).describe()

# Distribution of the management score variable

In [ ]:
(
    ggplot(df, aes(x="management"))
    + geom_histogram(
        aes(y="stat(count)/sum(stat(count))"),
        fill=color[0],
        color="white",
        binwidth=0.25,
        boundary=0,
        closed="left",
    )
    + labs(x="Management score", y="Percent")
    + scale_x_continuous(expand=(0.01, 0.01), limits=(1, 5))
    + scale_y_continuous(
        expand=(0.00, 0.00),
        limits=(0, 0.25),
        breaks=seq(0, 0.26, by=0.05),
        labels=percent_format(),
    )
    + theme_bw()
)


<div class="alert alert-info">
<h3> Your turn: Plot the distribution of employment </h3>
    
- variable= the number of employee (`emp_firm`)
- What type of graph will you use? 
- hint: you can use the `ggplot` function from `plotline` (cf. notebook `2.3-plotnine_intro`)
- specify meaningful y-axis & x-axis labels
    
</div>


(a) Number of employees

In [ ]:
(
    ggplot(df, aes(x="emp_firm", y="stat(count)/sum(stat(count))"))
    + geom_histogram(
        fill=color[0], color="white", binwidth=200, boundary=0, closed="left"
    )
    + labs(x="Firm size (employment)", y="Percent")
    + scale_x_continuous(
        expand=(0.01, 0.01), limits=(0, 5000), breaks=seq(0, 5001, by=1000)
    )
    + scale_y_continuous(
        expand=(0.00, 0.00),
        limits=(0, 0.3),
        breaks=seq(0, 0.5, by=0.05),
        labels=percent_format(),
    )
    + theme_bw()
)


<div class="alert alert-info">
<h3> Your turn: Plot the distribution of log(#employment) </h3>
    
1. Create such a variable and name it `lnemp`
2. Plot the distribution
</div>


(b) Natural log of number of employees

In [ ]:
df["emp3bins"] = np.where(df["emp_firm"] < 200, 1, pd.np.nan)
df["emp3bins"] = np.where(
    (df["emp_firm"] >= 200) & (df["emp_firm"] < 1000), 2, df["emp3bins"]
)
df["emp3bins"] = np.where(df["emp_firm"] >= 1000, 3, df["emp3bins"])

In [ ]:
df["emp3bins"].describe()


### Figure: Quality of specific management practices by three bins of firm size: conditional probabilities

(a) Lean management

In [ ]:
df1 = (
    df.filter(["emp3bins", "lean1"])
    .groupby(["emp3bins", "lean1"])
    .agg(Count=("emp3bins", "size"))
    .reset_index()
)
df1["Group_count"] = df1.groupby("emp3bins")["Count"].transform("sum")
df1["Percent"] = df1["Count"] / df1["Group_count"]
df1["lean1"] = pd.Categorical(
    df1["lean1"], categories=sorted(set(df1["lean1"]), reverse=True)
)
df1["emp3bins"] = pd.Categorical(
    df1["emp3bins"], categories=sorted(set(df1["emp3bins"]), reverse=True)
)

In [ ]:
(
    ggplot(df1, aes(x="emp3bins", y="Percent", fill="lean1"))
    + geom_bar(
        stat="identity", position="fill", width=0.6, color="white", size=0.5, alpha=0.8
    )
    + scale_y_continuous(
        expand=(0, 0),
        limits=(0, 1),
        breaks=seq(0, 1.1, by=0.2),
        labels=percent_format(),
    )
    + labs(x="Firm size (employment), 3 bins", y="Percent")
    + theme_bw()
    + theme(legend_position="right")
    + scale_fill_manual(
        values=(color[2], color[0], color[4], color[1], color[3]), name=" "
    )
)


(b) Performance tracking

In [ ]:
df1 = (
    df.filter(["emp3bins", "perf2"])
    .groupby(["emp3bins", "perf2"])
    .agg(Count=("emp3bins", "size"))
    .reset_index()
)
df1["Group_count"] = df1.groupby("emp3bins")["Count"].transform("sum")
df1["Percent"] = df1["Count"] / df1["Group_count"]
df1["perf2"] = pd.Categorical(
    df1["perf2"], categories=sorted(set(df1["perf2"]), reverse=True)
)
df1["emp3bins"] = pd.Categorical(
    df1["emp3bins"], categories=sorted(set(df1["emp3bins"]), reverse=True)
)

In [ ]:
(
    ggplot(df1, aes(x="emp3bins", y="Percent", fill="perf2"))
    + geom_bar(
        stat="identity", position="fill", width=0.6, color="white", size=0.5, alpha=0.8
    )
    + scale_y_continuous(
        expand=(0, 0),
        limits=(0, 1),
        breaks=seq(0, 1.1, by=0.2),
        labels=percent_format(),
    )
    + labs(x="Firm size (employment), 3 bins", y="Percent")
    + theme_bw()
    + theme(legend_position="right")
    + scale_fill_manual(
        values=(color[2], color[0], color[4], color[1], color[3]), name=" "
    )
)

In [ ]:
# Bin scatters avg score by employment bins

# Option 1: create 3 bins as defined by thresholds

# Summary

df.groupby("emp3bins")["emp_firm"].agg(["min", "max", "mean", "median", np.std, "size"])

In [ ]:
# Recode employee bins
df["emp3bins"] = df["emp3bins"].replace(1, 150).replace(2, 600).replace(3, 3000)


In [ ]:
# Summary
df.groupby("emp3bins")["emp_firm"].agg(["min", "max", "mean", "median", np.std, "size"])

In [ ]:
# Generate variables by mean
df1 = (
    df.groupby("emp3bins").agg(management_emp3bins=("management", "mean")).reset_index()
)
df1


### Figure 4.4 Mean management quality score and firm size

(a) Three bins of employment

In [ ]:
(
    ggplot(df1, aes(x="emp3bins", y="management_emp3bins"))
    + geom_point(size=2, color=color[2], fill=color[0], alpha=0.8, na_rm=True)
    + scale_y_continuous(
        expand=(0.01, 0.01), limits=(2.4, 3.4), breaks=seq(2.4, 3.5, by=0.2)
    )
    + scale_x_continuous(
        expand=(0.01, 0.01), limits=(0, 3000), breaks=seq(0, 3001, by=500)
    )
    + labs(x="Firm size (employment), 3 bins", y="Management score")
    + theme_bw()
)


In [ ]:
df["emp10bins"] = pd.qcut(df["emp_firm"], 10)


In [ ]:
# Summary
df_summary = (
    df.filter(["emp_firm", "emp10bins"])
    .groupby("emp10bins")
    .agg(["min", "max", "mean", "median", np.std, "size"])
)
df_summary


In [ ]:
# Recode with bin means
df = df.replace({"emp10bins": df_summary["emp_firm"]["mean"].to_dict()})


In [ ]:
df.groupby("emp10bins")["emp_firm"].agg(
    ["min", "max", "mean", "median", np.std, "size"]
)

In [ ]:
# Generate variables by mean
df1 = (
    df.groupby("emp10bins")
    .agg(management_emp10bins=("management", "mean"))
    .reset_index()
    .assign(emp10bins = lambda x: x["emp10bins"].astype(int))
)
df1


(b) Ten bins of employment

In [ ]:
(
    ggplot(df1, aes(x="emp10bins", y="management_emp10bins"))
    + geom_point(size=2, color=color[2], fill=color[0], alpha=0.8, na_rm=True)
    + scale_y_continuous(
        expand=(0.01, 0.01), limits=(2.5, 3.5), breaks=seq(2.5, 3.6, by=0.25)
    )
    + scale_x_continuous(
        expand=(0.01, 0.01), limits=(0, 3500), breaks=seq(0, 3501, by=500)
    )
    + labs(x="Firm size (employment), 10 bins", y="Management score")
    + theme_bw()
)


### Figure 4.5 The joint distribution of the management quality score and firm size

(a) By employment

In [ ]:
(
    ggplot(df, aes(x="emp_firm", y="management"))
    + geom_point(color=color[0], size=1.5, alpha=0.8, show_legend=False, na_rm=True)
    + scale_x_continuous(
        expand=(0.01, 0.01), limits=(0, 5000), breaks=seq(0, 5001, by=1000)
    )
    + scale_y_continuous(expand=(0.01, 0.01), limits=(1, 5), breaks=seq(1, 6, 1))
    + labs(x="Firm size (employment)", y="Management score")
    + theme_bw()
)


(b) By log employment

In [ ]:
df["lnemp"] = np.log(df["emp_firm"])

(
    ggplot(df, aes(x="lnemp", y="management"))
    + geom_point(color=color[0], size=1.5, alpha=0.8, show_legend=False, na_rm=True)
    + scale_x_continuous(expand=(0.01, 0.01), limits=(4, 9), breaks=seq(4, 10, by=1))
    + scale_y_continuous(expand=(0.01, 0.01), limits=(1, 5), breaks=seq(1, 6, 1))
    + labs(x="Firm size (ln(employment))", y="Management score")
    + theme_bw()
)


### Figure 4.6 Conditional summary statistics of the management score by bins of firm size

In [ ]:
df = df.replace({"emp3bins": {150: "Small", 600: "Medium", 3000: "Large"}})
df['emp3bins'] = pd.Categorical(df['emp3bins'], categories=sorted(set(df['emp3bins']), reverse=True))

Box plots

In [ ]:
(
    ggplot(df, aes(x="emp3bins", y="management", group="emp3bins"))
    + stat_boxplot(
        geom="errorbar",
        width=0.5,
        color=(color[1], color[0], color[2]),
        size=0.5,
        na_rm=True,
    )
    + geom_boxplot(
        color=(color[1], color[0], color[2]), size=0.5, width=0.5, na_rm=True
    )
    + labs(x="Firm size (employment), 3 bins", y="Management score")
    + scale_y_continuous(expand=(0.01, 0.01), limits=(1, 5), breaks=seq(1, 5, 1))
    + theme_bw()
)

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
sns.set_style("whitegrid")

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(
    data=df, x="emp3bins", y="management", width=0.5, ax=ax, flierprops={"marker": "o"}
)

box_line_col = [color[1], color[0], color[2]]
for i, box_col in enumerate(box_line_col):
    mybox = ax.patches[i]
    mybox.set_facecolor(mpl.colors.to_rgba(box_col, 0.5))
    mybox.set_edgecolor(box_col)
    for j in range(i * 6, i * 6 + 6):
        line = ax.lines[j]
        line.set_color(box_col)
        line.set_mfc(mpl.colors.to_rgba(box_col, 0.5))
        line.set_mec(mpl.colors.to_rgba(box_col, 0.5))

plt.ylabel("Management score", size=12)
plt.xlabel("Firm size (employment), 3 bins", size=12)
plt.yticks(seq(1, 5, 1))
add_margin(ax, x=0.1, y=0.01)
plt.show()

Violin plot

In [ ]:
(
    ggplot(
        df,
        aes(x="emp3bins", y="management", color="emp3bins", group="emp3bins"),
    )
    + geom_violin(size=0.3, alpha=0.5, trim=False, show_legend=False, na_rm=True)
    + geom_boxplot(size=0.5, width=0.2, alpha=0.3, na_rm=True)
    + labs(x="Firm size (employment), 3 bins", y="Management score")
    + scale_color_manual(values=(color[1], color[0], color[2]))
    + scale_y_continuous(expand=(0.01, 0.01), limits=(0, 6), breaks=seq(0, 6, 1))
    + guides(fill=False, color=False)
    + theme_bw()
)

In [ ]:
fig, ax = plt.subplots()
sns.violinplot(
    data=df,
    x="emp3bins",
    y="management",
    ax=ax,
    flierprops={"marker": "o"},
    width=1,
    linewidth=0.8,
    inner=None,
)

violin_line_col = [color[1], color[0], color[2]]
for i, violin_col in enumerate(violin_line_col):
    mybox = ax.collections[i]
    mybox.set_facecolor(mpl.colors.to_rgba(violin_col, 0.3))
    mybox.set_edgecolor(violin_col)

sns.boxplot(
    data=df,
    x="emp3bins",
    y="management",
    ax=ax,
    width=0.23,
    showcaps=False,
    flierprops={"marker": "o"},
)

box_line_col = [color[1], color[0], color[2]]
for i, box_col in enumerate(box_line_col):
    mybox = ax.patches[i]
    mybox.set_facecolor(mpl.colors.to_rgba(box_col, 0.4))
    mybox.set_edgecolor(box_col)
    for j in range(i * 4, i * 4 + 4):
        line = ax.lines[j]
        line.set_color(box_col)
        line.set_mfc(mpl.colors.to_rgba(box_col, 0.4))
        line.set_mec(mpl.colors.to_rgba(box_col, 0.4))

plt.ylabel("Management score", size=12)
plt.xlabel("Firm size (employment), 3 bins", size=12)
plt.yticks(seq(0, 6, 1))
add_margin(ax, x=0.1, y=0.01)
plt.show()

In [ ]:
# Correlation
df["management"].corr(df["emp_firm"])

In [ ]:
# by industry
df.loc[df["sic"] <= 21, "industry_broad"] = "food_drinks_tobacco"
df.loc[
    ((df["sic"] >= 22) & (df["sic"] <= 23)) | (df["sic"] == 31), "industry_broad"
] = "textile_apparel_leather_etc"
df.loc[(df["sic"] >= 24) & (df["sic"] <= 27), "industry_broad"] = "wood_furniture_paper"
df.loc[(df["sic"] >= 28) & (df["sic"] <= 30), "industry_broad"] = "chemicals_etc"
df.loc[(df["sic"] >= 32) & (df["sic"] < 35), "industry_broad"] = "materials_metals"
df.loc[(df["sic"] >= 35) & (df["sic"] < 37), "industry_broad"] = "electronics"
df.loc[df["sic"] == 37, "industry_broad"] = "auto"
df.loc[df["sic"] >= 38, "industry_broad"] = "other"

In [ ]:
df["industry_broad"].value_counts()


In [ ]:
# Correlation
df.groupby("industry_broad")["management", "emp_firm"].corr().iloc[0::2, -1]


In [ ]:
(
    df.loc[df["industry_broad"].notna(), ["management", "industry_broad"]]
    .groupby("industry_broad")
    .agg(
        Min=("management", min),
        Max=("management", max),
        SD=("management", np.std),
        Median=("management", "median"),
        n=("management", "size"),
    )
    .round(3)
)

In [ ]:
(
    df.loc[df["industry_broad"].notna(), ["emp_firm", "industry_broad"]]
    .groupby("industry_broad")
    .agg(
        Min=("emp_firm", min),
        Max=("emp_firm", max),
        SD=("emp_firm", np.std),
        Median=("emp_firm", "median"),
        n=("emp_firm", "size"),
    )
    .round(3)
)

In [ ]:
# Correlation
cor = (
    df.groupby("industry_broad")["management", "emp_firm"]
    .corr()
    .iloc[0::2, -1]
    .reset_index()
    .drop(["level_1"], axis=1)
    .set_index("industry_broad")
    .rename({"emp_firm": "correlation"}, axis=1)
)
cor.round(3)

In [ ]:
table41 = (
    df.filter(["emp_firm", "industry_broad", "management"])
    .groupby("industry_broad")
    .agg(Mean=("management", "mean"), Obs=("management", "size"))
)
table41["Corr"] = cor["correlation"]

In [ ]:
table41.index = [
    "Auto",
    "Chemicals",
    "Machinery, equipment, electronics",
    "Food, drinks, tobacco",
    "Materials, metals",
    "Textile, apparel, leather",
    "Wood, furniture, paper",
    "Other",
]
table41.round(3)


In [ ]:
last_row = (
    table41.groupby(lambda _: True)
    .agg(Mean=("Mean", "mean"), Obs=("Obs", "sum"), Corr=("Corr", "mean"))
    .reset_index(drop=True)
)
last_row.index = ["All"]
table41 = table41.append(last_row)

In [ ]:
table41 = table41.filter(["Corr", "Mean", "Obs"]).reset_index()
table41.columns = [
    "Industry",
    "Management - employment correlation",
    "Management score",
    "Observations",
]

In [ ]:
table41.round(2)
